In [1]:
import sys
print(sys.version)
import numpy as np
import ROOT
from array import *
import matplotlib.pyplot as plt
import math
from scipy.optimize import brentq

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Welcome to JupyROOT 6.30/04


In [2]:
def resultsTOtxt(arr,  filename):
#arr is the concatenated and transposed array of all the arrays wished to be written
#arrDescription a string describing and is each collumn of the file
#writes out each array concatenated in arr as an collumn in the file
    file = open(filename, 'w')
    np.savetxt(filename, arr, delimiter=' ')
    file.close()

In [3]:
def ReadNuclearModelSamples():
    fatm_hA_BR = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/atm_hA_BR.root')
    fbdm_m05_b1p1 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m05_b1p1_00a.root')    
    fbdm_m05_b1p5 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m05_b1p5_00a.root')             
    fbdm_m05_b10 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m05_b10_00a.root')     
    fbdm_m10_b1p1 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m10_b1p1_00a.root')
    fbdm_m10_b1p5 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m10_b1p5_00a.root')
    fbdm_m10_b10 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m10_b10_00a.root')
    fbdm_m20_b1p1 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m20_b1p1_00a.root')
    fbdm_m20_b1p5 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m20_b1p5_00a.root')
    fbdm_m20_b10 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m20_b10_00a.root')
    fbdm_m40_b1p1 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m40_b1p1_00a.root')
    fbdm_m40_b1p5 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m40_b1p5_00a.root')
    fbdm_m40_b10 = ROOT.TFile.Open('/home/leoperes/ntuples_grid_newVertex/bdm_m40_b10_00a.root')
    samples = [fbdm_m05_b1p1, fbdm_m05_b1p5, fbdm_m05_b10, fbdm_m10_b1p1, fbdm_m10_b1p5, fbdm_m10_b10, fbdm_m20_b1p1, fbdm_m20_b1p5, fbdm_m20_b10, fbdm_m40_b1p1, fbdm_m40_b1p5, fbdm_m40_b10]
    return samples


In [16]:
def GetNpArraysFromTrees(tree, branch_name):
    estimate = 3e6 #amount of numbers getting from branch
    tree.SetEstimate(int(estimate))
    result_hist = tree.Draw(branch_name, "", "goff")
    result = np.asarray(np.frombuffer(tree.GetV1(), dtype=np.float64, count=result_hist))
    result = result.copy()
    return result

In [17]:
def CVNCut(CVN_CutProb,CVN_SignalReco,CVN_Bg,CosSun_SignalValidCVN,AtmNuCosSun):
    nSignalEvents = 0 
    nBGEvents = 0
    EventsSignalCut = CVN_SignalReco[CVN_SignalReco > CVN_CutProb]
    EventsBgCut = CVN_Bg[CVN_Bg > CVN_CutProb] 
    CosSunSignal_CVNCut = CosSun_SignalValidCVN[CVN_SignalReco > CVN_CutProb]
    CosSunBgCVNCut = AtmNuCosSun[CVN_Bg > CVN_CutProb]
    return CosSunSignal_CVNCut, CosSunBgCVNCut, EventsSignalCut, EventsBgCut

In [19]:
def AngularCutBothSides(CVNScore_NC,BDTScore, nSignalSimulatedEvts): 
    
    results = []
    SprimeMin = 999999999
    bestCutRight = 9999
    bestCutLeft = 9999
    bestEff = 9999
    bestBkg = 9999
    bestBkgErr = 999
    Sprime = 999
    CosSunSignal = []
    CosSunAtm = []
    
    CosSunSignalCVNCut, CosSunBgCVNCut, SignalEvts, BgEvts = CVNCut(CVNScore_NC, CVN_Signal_Valid, CVN_Atm_Valid, CosSun_Signal_wCVN, CosSun_Atm_wCVN)
    
    BackgroundScale = 4 * 10 * RatioRecoAtm *2235.8 /nSimulatedEvtsAll_atm
    # = 40kt * 10 years * reco/simulated ratio * # atm nu expected for 10kt y * inside/out cvn cut ratio / #simulated atm nu events 

    CosCutsToRight = np.linspace(0,1.00,40)
    CosCutsToLeft = np.linspace(0,-1.00,40)
    
    CosSunEvts = np.concatenate((CosSunSignalBDTCut,CosSunSignalCVNCut), axis=0)
    CosSunAtmEvts = np.concatenate((CosSunAtmBDTCut, CosSunBgCVNCut), axis=0)
    for CosCutRight in CosCutsToRight:

        for CosCutLeft in CosCutsToLeft:
            nReco = 0.0001
            nBG = 0.00001
            nRecoToTheRight = np.sum(CosCutRight < CosSunEvts)
            nRecoToTheLeft = np.sum(CosCutLeft > CosSunEvts)
            nReco = nRecoToTheRight + nRecoToTheLeft
            
            nBGToTheRight = np.sum(CosCutRight < CosSunAtmEvts)
            nBGToTheLeft = np.sum(CosCutLeft > CosSunAtmEvts)
            nBG = nBGToTheRight + nBGToTheLeft
            
            rReco = nReco/nSignalSimulatedEvts  
            bEvents = nBG*BackgroundScale
            
            #print(rReco,nReco,nSignalSimulatedEvts)
            Sprime = 25./ (2.*rReco) + math.sqrt( 25.*bEvents/ (rReco*rReco) + 625./(4.*rReco*rReco) )
            
            if Sprime < SprimeMin:
                SprimeMin = Sprime
                bestCutRight = CosCutRight
                bestCutLeft = CosCutLeft
                bestEff = rReco
                bestBkg = bEvents
                bestBkgErr = math.sqrt( nBG *BackgroundScale)
                BestCVNCut = CVNScore_NC
                BestBDTCut = BDTScore
                CosSunSignal = CosSunEvts
                CosSunAtm = CosSunAtmEvts
    results = [SprimeMin, bestCutLeft, bestCutRight, bestEff, bestBkg, bestBkgErr, CVNScore_NC, BDTScore, CosSunEvts, CosSunAtmEvts]
    return results



In [ ]:
infiles_list = ReadNuclearModelSamples()
for SAMPLE_INDEX in range(0,len(infiles_list)):
    intrees_reco = [lambda arg=infile: arg.Get("ana/Atm") for infile in infiles_list]
    intrees_simu = [lambda arg=infile: arg.Get("ana/AllEvents") for infile in infiles_list]
    reco_entries = [intree().GetEntries() for intree in intrees_reco]
    simu_entries = [intree().GetEntries() for intree in intrees_simu]

In [20]:
CVN_Atm = GetNpArraysFromTrees(fTree_atm,"CVN_NCScore")
CosSun_Atm = 2*np.random.random_sample((nSimulatedEvtsReco_atm,))-1

CVN_Atm[CVN_Atm==-1] = 1.


Sensitivity_Info = open("Sensitivity_Info_00a.txt", "a")
Sensitivity_results = open("Sensitivity_results_00a.txt","a")
Eff_Bkg_index =open("Eff_Bkg_index_00a.txt","a")
CVN_Cuts = np.linspace(0.2,1.00,35)

index = 0

Sensitivity_Info.write("################# SAMPLE INFO #######################"+"\n")
Sensitivity_Info.write("\n")
Sensitivity_Info.write("Number of Atmospheric Neutrinos Events Simulated: " +str(nSimulatedEvtsAll_atm)+"\n")
Sensitivity_Info.write("Number of Atmospheric Neutrinos Events Reconstructed: " +str(nSimulatedEvtsReco_atm)+"\n")
Sensitivity_Info.write("Ratio of Atmospheric Neutrinos Events Reconstructed: " +str(RatioRecoAtm)+"\n")
Sensitivity_Info.write("\n")

resultsTOtxt(CVN_Atm, 'CVN_Atm_hA_BR.txt')
resultsTOtxt(CosSun_Atm, 'CosSun_Atm_hA_BR.txt')

Eff_Bkg_index.write("SigStrength\tNegCosCut\tPosCosCut\tbestEff\tBkgExpect\tBkgError\tCVNCut\tBDTCut\n")


73

                         : Booking "TMVAClassification" of type "BDT" from /home/leoperes/Dropbox/BoostedDM/Final_Analysis/dataset_v02/weights/TMVAClassification_BDTAtmCCNC.weights.xml.
                         : Reading weight file: /home/leoperes/Dropbox/BoostedDM/Final_Analysis/dataset_v02/weights/TMVAClassification_BDTAtmCCNC.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTAtmCCNC" of type: "BDT"
                         : Rebuilding Dataset Default


In [23]:
for sig_tree in signalTreesList:
    
    CVN_Signal = GetNpArraysFromTrees(sig_tree,"CVN_NCScore")
    CosSun_Signal = GetNpArraysFromTrees(sig_tree,"DiffCosAngleTotalMom")
    BDTResponseSignal = GetBDTResponse(sig_tree)
    CVN_Signal_Valid = CVN_Signal[CVN_Signal>-1]
    CosSun_Signal_wCVN = CosSun_Signal[CVN_Signal> -1]
    CosSun_Signal_NoCVN = CosSun_Signal[CVN_Signal == -1]

    
    bestResultsTwoAngCuts = []
    SpMinTwoAngCuts = 99999999.9
    bestCVNCutTwoCut = -3

    Sensitivity_Info.write("Sample: "+str(sig_tree.GetCurrentFile().GetName()[28:-5])+"\n")
    Sensitivity_Info.write("Number of Boosted DM Events Simulated: " +str(nSimulatedEvtsList[index])+"\n")
    Sensitivity_Info.write("Number of Boosted DM Events Reconstructed: " + str(nSimulatedEvtsRecoList[index])+"\n")
    Sensitivity_Info.write("Ratio of Boosted DM Events Reconstructed: " +str(float(nSimulatedEvtsRecoList[index])/nSimulatedEvtsList[index])+"\n")
    Sensitivity_Info.write("\n")

    Sensitivity_results.write("Sample: "+str(sig_tree.GetCurrentFile().GetName()[28:-5])+"\n")
    Sensitivity_results.write("SigStrength\tNegCosCut\tPosCosCut\tbestEff\tBkgExpect\tBkgError\tCVNCut\tBDTCut\n")
    for CVNScore in CVN_Cuts:
        for BDT_Score in BDT_Cuts:
            resultsTwoAngCuts = AngularCutBothSides(CVNScore, BDT_Score, nSimulatedEvtsList[index])
            if resultsTwoAngCuts[0] < SpMinTwoAngCuts:
                bestResultsTwoAngCuts = resultsTwoAngCuts
                bestCVNCutTwoCut = CVNScore
                SpMinTwoAngCuts = resultsTwoAngCuts[0]
                
                Sensitivity_results.write(str(bestResultsTwoAngCuts[0])+"\t"+str(bestResultsTwoAngCuts[1])+"\t"+str(bestResultsTwoAngCuts[2])+"\t"+str(bestResultsTwoAngCuts[3])+"\t"+str(bestResultsTwoAngCuts[4])+"\t"+str(bestResultsTwoAngCuts[5])+"\t"+str(bestResultsTwoAngCuts[6])+"\t"+str(bestResultsTwoAngCuts[7])+"\n")
                

    Sensitivity_results.write(str(" ======= BEST CUTS ========")+"\n")
    Sensitivity_results.write(str(bestResultsTwoAngCuts[0])+"\t"+str(bestResultsTwoAngCuts[1])+"\t"+str(bestResultsTwoAngCuts[2])+"\t"+str(bestResultsTwoAngCuts[3])+"\t"+str(bestResultsTwoAngCuts[4])+"\t"+str(bestResultsTwoAngCuts[5])+"\t"+str(bestResultsTwoAngCuts[6])+"\t"+str(bestResultsTwoAngCuts[7])+"\n")   
    np.savetxt(Eff_Bkg_index, np.array(bestResultsTwoAngCuts[0:8]).flatten(), fmt="%.3f", newline = " ",delimiter=', ')
    Eff_Bkg_index.write("\n")
    Sensitivity_results.write("\n")
    print('\n')
    print(index)
    print("################# BEST ANGULAR AND CVN CUTS INFO #######################")
    print("Optimal Signal Strength: " + '{:.0f}'.format(bestResultsTwoAngCuts[0]))
    print("Optimal CVN Cut: " +'{:.3f}'.format(bestResultsTwoAngCuts[6]))
    print("Optimal BDT Cut: " +'{:.3f}'.format(bestResultsTwoAngCuts[7]))
    print("Optimal Positive Cossine Angular Cut: "  + '{:.3f}'.format(bestResultsTwoAngCuts[2]))
    print("Optimal Negative Cossine Angular Cut: " +  '{:.3f}'.format(bestResultsTwoAngCuts[1]))
    print("Signal Efficiency: " + '{:.3f}'.format(bestResultsTwoAngCuts[3]))
    print("")
    print("Number of expected Atm Nu events: " + '{:.1f}'.format(bestResultsTwoAngCuts[4]) + ", statistical uncertainty: " + '{:.1f}'.format(bestResultsTwoAngCuts[5]))
    print('\n')
    #h.write('{:.3f}'.format(bestResultsTwoAngCuts[3]),'{:.3f}'.format(bestResultsTwoAngCuts[4]),'{:.1d}'.format(index))
   
    resultsTOtxt(bestResultsTwoAngCuts[8], 'CosSunSignalAfterCuts_'+'{:.20s}'.format(SamplesNameList[index])+'.txt')
    resultsTOtxt(bestResultsTwoAngCuts[9], 'CosSunAtmAfterCuts_'+'{:.20s}'.format(SamplesNameList[index])+'.txt')
    resultsTOtxt(CVN_Signal, 'CVN_'+'{:.20s}'.format(SamplesNameList[index])+'.txt')
    resultsTOtxt(BDTResponseSignal, 'BDT_'+'{:.20s}'.format(SamplesNameList[index])+'.txt')
    resultsTOtxt(CosSun_Signal, 'CosSun_'+'{:.20s}'.format(SamplesNameList[index])+'.txt')
    index = index + 1

ValueError: I/O operation on closed file.

                         : Booking "TMVAClassification" of type "BDT" from /home/leoperes/Dropbox/BoostedDM/Final_Analysis/dataset_v02/weights/TMVAClassification_BDTAtmCCNC.weights.xml.
                         : Reading weight file: /home/leoperes/Dropbox/BoostedDM/Final_Analysis/dataset_v02/weights/TMVAClassification_BDTAtmCCNC.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTAtmCCNC" of type: "BDT"
                         : Rebuilding Dataset Default


In [22]:
Sensitivity_Info.close()
Sensitivity_results.close()
Eff_Bkg_index.close()